In [5]:
import torch
import torch.nn as nn
from transformers import ViTModel, ViTConfig

# Load the pretrained ViT model
model_name = "google/vit-base-patch16-384"
vit_model = ViTModel.from_pretrained(model_name)

# Define a self-attention classifier
class SelfAttentionClassifier(nn.Module):
    def __init__(self, embed_dim, num_heads, num_classes):
        super(SelfAttentionClassifier, self).__init__()
        self.self_attn = nn.MultiheadAttention(embed_dim, num_heads)
        self.classifier = nn.Linear(embed_dim, num_classes)

    def forward(self, x):
        attn_output, attn_weights = self.self_attn(x, x, x)
        logits = self.classifier(attn_output)
        return logits, attn_weights

# Attach the self-attention classifier to the ViT model
class ViTWithClassifier(nn.Module):
    def __init__(self, vit_model, num_heads, num_classes):
        super(ViTWithClassifier, self).__init__()
        self.vit_model = vit_model
        self.classifier = SelfAttentionClassifier(
            embed_dim=vit_model.config.hidden_size,
            num_heads=num_heads,
            num_classes=num_classes
        )

    def forward(self, x):
        outputs = self.vit_model(pixel_values=x)
        last_hidden_state = outputs.last_hidden_state  # shape: (batch_size, seq_len, hidden_size)
        # Exclude the class token and pass the patch embeddings to the classifier
        patch_embeddings = last_hidden_state[:, 1:, :]
        logits, attn_weights = self.classifier(patch_embeddings)
        return logits, attn_weights

# Initialize the model with the classifier
num_classes = 10  # Number of target classes for your classification task
num_heads = 8  # Number of attention heads
model = ViTWithClassifier(vit_model, num_heads, num_classes)

c:\Users\remote desktop\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\remote desktop\AppData\Local\Programs\Python\Python312\Lib\site-packages\huggingface_hub\file_download.py:159: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\remote desktop\.cache\huggingface\hub\models--google--vit-base-patch16-384. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Develop